# Bibliotecas usadas

In [6]:
import numpy as np
import pandas as pd
from itertools import combinations

# Leer datos


In [7]:

datos = np.load('spotify.npy', encoding = "latin1",allow_pickle="False") # Cargar archivo


## Ordenamos las listas de reproduccion dentro de un arreglo

In [8]:
playlists=[]
for i in datos.item():
    datos.item()[i].sort()
    playlists.append(list((set(datos.item()[i]))))

### Mostramos los datos

In [9]:
playlists

[['See You Again',
  'Somebody To Love',
  'All The Small Things',
  "Leavin'",
  'Check Yes Juliet',
  'Right Where You Want Me - Radio Edit Version',
  'Sk8er Boi',
  'The Great Escape',
  'My Boo',
  'Yo (Excuse Me Miss)',
  "Stacy's Mom",
  'Where Is The Love?',
  'Closer',
  'Your Love Is My Drug',
  'Whatcha Say',
  'Lip Gloss',
  'Me & U',
  'Check On It - feat. Bun B and Slim Thug',
  'Buttons',
  'My Happy Ending',
  'Run It!',
  'A Thousand Miles',
  'Forever',
  'Dirty Little Secret',
  'Yeah!',
  'Everytime We Touch - Radio Edit',
  'Party In The U.S.A.',
  'How Do You Sleep? - Featuring Ludacris',
  'Lose Control (feat. Ciara & Fat Man Scoop)',
  'Baby',
  'Replay',
  'Year 3000',
  'Beautiful Soul',
  'Rock Your Body',
  'Say My Name',
  'Ice Box',
  'Hey Ya! - Radio Mix / Club Mix',
  'This Is Me',
  'Toxic',
  "It Wasn't Me",
  'Crazy In Love',
  "Jumpin', Jumpin'",
  'Livin on Sunday',
  'Paper Planes',
  'Beep',
  'Soak Up The Sun',
  'Mr. Brightside',
  'One Less Lon

# Algoritmo Apriori

## Funcion para crear el primer nivel, recorriendo todas las canciones

In [10]:
def primerNivel(datos,min):
  #Creamos un diccionario el cual almacenara las canciones y su indice de aparicion de estas
  listaCancionesTodas = {} 
  listaAparicion={}
  #Se calcula el suport minimo
  newMin=min*len(datos)
  for playlist in datos:
    for grupoCanciones in playlist:
      #Haciendo uso del diccionario contamos y agregamos el numero de playlist en el que estas salen
      listaCancionesTodas[grupoCanciones] = listaCancionesTodas.get(grupoCanciones, 0) + 1
      listaAparicion[grupoCanciones]=listaAparicion.get(grupoCanciones, [])+[datos.index(playlist)]
  listaCancionesPasan=[]
  listaAparicionPasan={}
  #Filtramos aquellos itemsets que pasen el suport minimo
  for dato in listaAparicion:
    if(len(listaAparicion[dato])>=newMin):
      listaCancionesPasan.append([dato])
      listaAparicionPasan[tuple([dato])]=listaAparicion.get(dato)
  lista1=sorted(listaAparicionPasan.items())
  return lista1


## Funcion usada para la creacion de un nuevo nivel a partir del anterior

In [21]:
def crearNivel(datosPrevios,tamañoDatos,min,nivel):
  #Se define el nuevo nivel
  nuevoNivel={}
  numDatosPrevios=len(datosPrevios)
  newMin=min*tamañoDatos # Es utilizado para realizar un solo calculo por nivel en lugar de dividir el numero de aparicion entre el numero de elementos
  listaAparicion={}
  #Recorremos la lista de 2 en 2 , para que compare estos y vea si poseen una diferencia en el ultimo caracter
  for i in range(numDatosPrevios):
    for j in range(i+1,numDatosPrevios):
      L1 = datosPrevios[i][0] [: nivel]
      L2 = datosPrevios[j][0] [: nivel] 
      L11=sorted(L1)
      L22=sorted(L2)
      #Se pregunta si las listas son iguales a excepcion del ultimo caracter
      if L11==L22:
        #Si son iguales se genera un nuevo elemento el cual sera analizado a continuacion
        nuevoElemento=set(datosPrevios [i][0]) | set(datosPrevios [j][0])
        conjuntoAparicion=set(datosPrevios[i][1])&set(datosPrevios[j][1]) # Se trabaja con listas que cuentan en el lugar en el que estos se ubican
        nuevoNivel[tuple(nuevoElemento)]=list(conjuntoAparicion) 
  #En listaPasan se filtran aquellos nuevos itemsets que pasen por el minimo suport
  listaPasan={}
  for key in nuevoNivel:
    if(len(nuevoNivel[key])>=newMin):
      listaPasan[tuple(sorted(key))]=sorted(nuevoNivel.get(key))
      
  lista1=sorted(listaPasan.items())
  return lista1
    

## Creacion de los Itemsets con Apriori

In [12]:
def apriori(datos,suport):
    #Calculamos el primer nivel de los datos es decir, cuando se agregan los itemsets de longitud 1
    inicio = primerNivel(datos,suport)
    #Se guarda el tamaño de los datos para ser calculados una unica vez
    tamañoDatos=len(datos)
    #En la linea Itemsets se almacenaran todos los itemsets que hayan cumplid con el soporte minimo
    ListaItemsets = [inicio]
    k = 0
    indicador=True
    #Se realizan las iteraciones hasta que encuentre un elemento nulo
    while (indicador):
        nivelK = crearNivel(ListaItemsets[k],tamañoDatos, suport,k)
        if(len(nivelK)>0):
            ListaItemsets.append(nivelK)
        else:
            indicador=False
        k += 1
    return ListaItemsets

### Encontramos los itemsets frecuentes

In [13]:
itemsets=apriori(playlists,0.0075)

### Mostramos los itemsets encontrados y cuantos niveles tiene


In [14]:
for i in range( len(itemsets)):
    print("- Elementos del nivel",i+1,"son ",len(itemsets[i]))

- Elementos del nivel 1 son  1333
- Elementos del nivel 2 son  523
- Elementos del nivel 3 son  161
- Elementos del nivel 4 son  6


### Mostramos el soporte de los elementos

In [15]:
for i in range(len(itemsets)):
    print("========== Itemsets de ",i+1,"elementos ==========")
    for elemento in itemsets[i]:
        print("-",elemento[0],"tienen un soporte de: ",len(elemento[1])/len(playlists))

========== Itemsets de  1 elementos ==========
- ('$ave Dat Money (feat. Fetty Wap & Rich Homie Quan)',) tienen un soporte de:  0.0089
- ("'Till I Collapse",) tienen un soporte de:  0.0169
- ("(Don't Fear) The Reaper",) tienen un soporte de:  0.0091
- ("(I Can't Get No) Satisfaction - Mono Version / Remastered 2002",) tienen un soporte de:  0.0077
- ("(Sittin' On) The Dock Of The Bay",) tienen un soporte de:  0.0075
- ('0 To 100 / The Catch Up',) tienen un soporte de:  0.0161
- ('1-800-273-8255',) tienen un soporte de:  0.0264
- ('10 Bands',) tienen un soporte de:  0.0101
- ('1901',) tienen un soporte de:  0.0085
- ('1985',) tienen un soporte de:  0.0107
- ('2 Phones',) tienen un soporte de:  0.0147
- ('23',) tienen un soporte de:  0.0088
- ('24K Magic',) tienen un soporte de:  0.0167
- ('2U (feat. Justin Bieber)',) tienen un soporte de:  0.0116
- ('4 AM',) tienen un soporte de:  0.0122
- ('6 Foot 7 Foot',) tienen un soporte de:  0.014
- ('679 (feat. Remy Boyz)',) tienen un soporte de:

## Reglas de asociacion

In [16]:
def getReglasdeAsociacion(itemsetsFrecuentes,minConfianza,minLift):
  Reglas=[]
  confianzas=[]
  lifts=[]
  #empezamos a recorrer los itemsets de manera inversa para tener los itemsets de mayor tamaño
  # diccionario de suport
  suportDict={}
  for nivel in itemsetsFrecuentes:
    for elemento in nivel:
      suportDict[elemento[0]]=len(elemento[1])/10000
  tamañoMax=len(itemsets)-1
  
  while(tamañoMax>=1):
    for itemset in itemsets[tamañoMax]:
      combinaciones=list(combinations(itemset[0],tamañoMax))
      for combinacion in combinaciones:
        confianza=suportDict[itemset[0]]/suportDict[combinacion]
        diferencia=tuple(set(itemset[0]).difference(set(combinacion)))
        lift=suportDict[itemset[0]]/(suportDict[combinacion]*suportDict[diferencia])
        #print(lift)
        if(confianza>=minConfianza and lift>minLift):
          Reglas.append([combinacion,diferencia])
          confianzas.append(confianza)
          lifts.append(lift)
    tamañoMax-=1
  listaFinal=[]
  for i in range(len(Reglas)):
    aux=[Reglas[i],confianzas[i],lifts[i]]
    listaFinal.append(aux)
  return listaFinal



## Calculamos las reglas de asociacion

In [19]:
reglas=getReglasdeAsociacion(itemsets,0.83,1)

### Mostramos las reglas junto a su confianza y lift

In [20]:
for i in range(len(reglas)):
    print("====== Regla ",i+1," ======")
    print("regla: ",reglas[i][0][0],"->",reglas[i][0][1])
    print("confianza: ",reglas[i][1])
    print("lift: ",reglas[i][2])

====== Regla  1  ======
regla:  ('Congratulations', 'Mask Off', 'goosebumps') -> ('HUMBLE.',)
confianza:  0.8315789473684212
lift:  17.8834182229768
====== Regla  2  ======
regla:  ('Mask Off', 'Tunnel Vision', 'XO TOUR Llif3') -> ('HUMBLE.',)
confianza:  0.875
lift:  18.817204301075268
====== Regla  3  ======
regla:  ('Mask Off', 'XO TOUR Llif3', 'goosebumps') -> ('HUMBLE.',)
confianza:  0.8415841584158417
lift:  18.098584051953583
====== Regla  4  ======
regla:  ('Bad and Boujee (feat. Lil Uzi Vert)', 'DNA.') -> ('HUMBLE.',)
confianza:  0.885057471264368
lift:  19.03349400568533
====== Regla  5  ======
regla:  ('Bounce Back', 'DNA.') -> ('HUMBLE.',)
confianza:  0.941860465116279
lift:  20.255063765941483
====== Regla  6  ======
regla:  ('Congratulations', 'DNA.') -> ('HUMBLE.',)
confianza:  0.8857142857142856
lift:  19.047619047619044
====== Regla  7  ======
regla:  ('DNA.', 'Mask Off') -> ('HUMBLE.',)
confianza:  0.9090909090909092
lift:  19.550342130987293
====== Regla  8  ======
r

# Conclusiones e interpretaciones

Las reglas de asociacion encontradas muestran los siguientes resultados de los cuales se puede inferir 

* Dentro de las 10 reglas de asociacion, el patron que se repite en ellas es de que estas pertenecen al mismo artista
* Se tiene como regla el mismo tipo de canciones, las cuales son de cantantes afroamericanos
* El genero de las canciones es el rap
* La relacion que guarda la cancion DNA con HUMBLE , se debe a que estas dos canciones fueron hechas por el artista Kendrick Lamar en el 2017